# Sentiment Analysis sulle Recensioni di Yelp

La **Sentiment Analysis** è il processo di identificazione dell'emozione espressa in un testo, positiva o negativa.


In questo notebook useremo Spark e la sua MLlib per costruire un modello di Sentiment Analysis usando il dataset messo a disposizione da Yelp, una famossisima applicazione che permette di recensire locali e attività commerciali.

## Importazione delle librerie

In [1]:
import os
import shutil
import pandas as pd

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
from pyspark.sql.functions import count, max, col, year, sum

## Inizializzazione di Spark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()
from pyspark.sql.types import *

## Caricamento del dataset

Scarico da Kaggle lo .zip contenente il dataset.

In [ ]:
!kaggle datasets download yelp-dataset/yelp-dataset

Estraggo il dataset dal .zip

In [ ]:
shutil.unpack_archive('yelp-dataset.zip')

Carico il dataset.

In [5]:
yelp_df = spark.read.json('yelp_academic_dataset_review.json')

In [6]:
yelp_df.show(3)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|-MhfebM0QIsKt87iD...|   0|2015-04-15 05:21:16|    0|xQY8N_XvtGbearJ5X...|  2.0|As someone who ha...|     5|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|   0|2013-12-07 03:16:52|    1|UmFMZ8PyXZTY2Qcwz...|  1.0|I am actually hor...|     1|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|   0|2015-12-05 03:18:11|    0|LG2ZaYiOgpr2DK_90...|  5.0|I love Deagan's. ...|     1|V34qejxNsCbcgD8C0...|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
only showing top 3 rows



Esaminiamo questa tabella. Innanzitutto vediamo il numero di righe.

In [7]:
yelp_df.count()

8021122

Ora le colonne.

In [8]:
yelp_df.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

Abbiamo ben 9 colonne, che sono:

* **user_id**: identificativo del recensore
* **business_id**: identificato del business recensito
* **review_id**: identificativo della recensione
* **text**: testo della recensione
* **date**: data della recensione
* **stars**: valutazione dell'attività (da 1.0 a 5.0).
* **useful**: numero di utenti che hanno segnato la recensione come uile
* **cool**: numero degli utenti che hanno segnato la recensione come figa.
* **funny**: numero di utenti che hanno segnato la recensione come divertente.

In [9]:
yelp_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



Le uniche informazioni che a noi interessano sono il testo e la valutazione, creiamo un nuovo DataFrame con soltanto queste colonne.

In [10]:
reviews_df = yelp_df.select('text','stars')

In [11]:
reviews_df.show(5)

+--------------------+-----+
|                text|stars|
+--------------------+-----+
|As someone who ha...|  2.0|
|I am actually hor...|  1.0|
|I love Deagan's. ...|  5.0|
|Dismal, lukewarm,...|  1.0|
|Oh happy day, fin...|  4.0|
+--------------------+-----+
only showing top 5 rows



Cominciamo creando un modello utilizzando soltanto il 1% del dataset.

In [12]:
subreviews_df = reviews_df.sample(fraction=0.01, seed=0)

## Preprocessing del testo

Ora dobbiamo processare il testo delle recensioni per renderlo un buon input per una modello di machine learning. 

### Rimozione della punteggiatura

Genero una funzione per la rimozione della punteggiatura.

In [13]:
import string

def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

Utilizziamo la **funzione udf** (User Defined Function - Funzione Definita dall'Utente) per creare una funzione spark partendo da quella che abbiamo definito noi per la rimozione della punteggiatura.

In [14]:
from pyspark.sql.functions import udf, col

funz = udf(lambda s: remove_punct(s))

Fatto questo applichiamo la funzione alla colonna text, per rimuovere la punteggiatura da ogni recensione.

In [15]:
subreviews_df = subreviews_df.withColumn('text', funz(col('text')))

In [16]:
subreviews_df.show(5)

+--------------------+-----+
|                text|stars|
+--------------------+-----+
|I have been here ...|  4.0|
|So far one of the...|  5.0|
|Came here for hap...|  1.0|
|Bad 1st experienc...|  1.0|
|So many great thi...|  4.0|
+--------------------+-----+
only showing top 5 rows



N.B: avremmo potuto ottenere lo stesso effetto anche applicando direttamente un **decoratore**.

In [17]:
@udf()
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

subreviews_df.withColumn('remove_punct_text', remove_punct(col('text'))).show(5)

+--------------------+-----+--------------------+
|                text|stars|   remove_punct_text|
+--------------------+-----+--------------------+
|I have been here ...|  4.0|I have been here ...|
|So far one of the...|  5.0|So far one of the...|
|Came here for hap...|  1.0|Came here for hap...|
|Bad 1st experienc...|  1.0|Bad 1st experienc...|
|So many great thi...|  4.0|So many great thi...|
+--------------------+-----+--------------------+
only showing top 5 rows



### Tokenizzazione

Ora eseguiamo la **Tokenizzazione**, che ci serve per estrarre i Token dal testo, cioè le sue parti costituenti (le parole insomma). Per farlo possiamo usare la classe **Tokenizer** di MLlib.

In [18]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(subreviews_df)

In [19]:
words_df.show(5)

+--------------------+-----+--------------------+
|                text|stars|               words|
+--------------------+-----+--------------------+
|I have been here ...|  4.0|[i, have, been, h...|
|So far one of the...|  5.0|[so, far, one, of...|
|Came here for hap...|  1.0|[came, here, for,...|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|
|So many great thi...|  4.0|[so, many, great,...|
+--------------------+-----+--------------------+
only showing top 5 rows



### Rimozione StopWords

Ora abbiamo ogni recensione rappresentata da una lista di parole, molte di queste parole sono superflue e non portano informazioni utili ai fini della sentiment analysis. Tali parole sono dette **StopWords** ed è buona pratica rimuoverle, possiamo farlo utilizzando la classe **StopWordsRemover** di MLlib.



In [20]:
from pyspark.ml.feature import StopWordsRemover

stopwords = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = stopwords.transform(words_df)

In [21]:
words_df.show(5)

+--------------------+-----+--------------------+--------------------+
|                text|stars|               words|            filtered|
+--------------------+-----+--------------------+--------------------+
|I have been here ...|  4.0|[i, have, been, h...|[twice, nice, lai...|
|So far one of the...|  5.0|[so, far, one, of...|[far, one, best, ...|
|Came here for hap...|  1.0|[came, here, for,...|[came, happy, hou...|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|[bad, 1st, experi...|
|So many great thi...|  4.0|[so, many, great,...|[many, great, thi...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



### CountVectorizer [Bag of Words]

Ora, utilizzando la colonna 'filtered' come corpus testuale (ogni riga sarà un document), andiamo a generare il **vocabolario dei tokens**; nello specifico, ogni riga di questa nuova colonna che andiamo a generare sarà composta da:

* **numero di vocaboli distinti** presi da tutte le righe (sotto la colonna 'words'), dunque facenti parte del vocabolario generato
* array degli indici delle **parole (distinte) del vocabolario presenti nella riga** (sotto la colonna 'words')
* **numero di volte** in cui si ripresentano le parole del punto precedente

Questa tecnica ci consente di convertire una recensione in una lista di numeri (dato che in input a un modello di machine learning possiamo dare numeri e non parole).

In [22]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol='filtered', outputCol='features')
cv_model = cv.fit(words_df)
cv_df = cv_model.transform(words_df)

In [23]:
cv_df.show(1, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In realtà, nel modello **Bag of Words**, un document è rappresentato da un vettore avente lunghezza pari all'ampiezza del vocabolario e ogni suo elemento viene valorizzato con:

* **1** laddove la corrispondente parola del vocabolario è presente nel document
* **0** laddove questa corrispondenza non sussiste

Dunque se seguissimo questo apprioccio avremmo, in questo caso, 8021122 di vettori (righe dataset) contenenti ciascuno 101222 elementi (ampiezza dizionario), la maggior parte dei quali pari ovviamente a 0.
**Spark**, che tende sempre a ottimizzare l'utilizzo della memoria, utilizza un approccio differente (quello descritto sopra nei 3 punti) che consente di "salvare" tantissimo spazio ottenendo lo stesso risultato.

Ora scartiamo pure tutte le colonne intermedie che abbiamo creato tenendo soltanto quelle che ci serviranno per realizzare il modello, features e stars.

In [24]:
cv_df = cv_df.select(["features","stars"])
cv_df.show(5)

+--------------------+-----+
|            features|stars|
+--------------------+-----+
|(101222,[0,2,6,9,...|  4.0|
|(101222,[0,1,2,3,...|  5.0|
|(101222,[1,2,4,5,...|  1.0|
|(101222,[3,23,30,...|  1.0|
|(101222,[0,2,4,6,...|  4.0|
+--------------------+-----+
only showing top 5 rows



## Quali sono le recensioni negative ?

Come abbiamo già detto le recensioni sono accompagnate da una valutazione che va da 1.0 a 5.0 stelle.
Etichettiamo come
* **positive** le recensioni con una valutazione di almeno 3.5 stelle
* **negative** le recensioni con una valutazione inferiore alle 2.5 stelle.
* le recensioni con 3 stelle sono tendenzialmente **neutre**, quindi rimuoviamole dal dataframe.

In [25]:
from pyspark.sql.functions import when

cv_df = cv_df.filter(cv_df.stars != 3)
cv_df = cv_df.withColumn('label',when(cv_df['stars'] >= 3.5, 1).otherwise(0))

In [26]:
cv_df.show(5)

+--------------------+-----+-----+
|            features|stars|label|
+--------------------+-----+-----+
|(101222,[0,2,6,9,...|  4.0|    1|
|(101222,[0,1,2,3,...|  5.0|    1|
|(101222,[1,2,4,5,...|  1.0|    0|
|(101222,[3,23,30,...|  1.0|    0|
|(101222,[0,2,4,6,...|  4.0|    1|
+--------------------+-----+-----+
only showing top 5 rows



Così facendo abbiamo generato un dataset adatto a un modello di apprendimento automatico **supervisionato**

Utilizziamo il metodo *randomSplit* per generare il **train set** e il **test set**
* un DataFrame per l'addestramento del modello che conterrà il 70% degli esempi.
* un DataFrame per il testing del modello che conterrà il restante 30% degli esempi.

In [27]:
train_df, test_df = cv_df.randomSplit([0.7, 0.3])

Ora possiamo creare il modello, utilizziamo una Regressione Logistica.

In [28]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label")
# in caso di overfitting è possibile utilizzare il parametro 'regParam' per introdurre il meccanismo di regolarizzazione;
# infatti regParam è proprio il coefficiente di regolarizzazione
model = lr.fit(train_df)

Valutiamo il modello addestrato sul DataFrame di test.

In [29]:
evaluation = model.evaluate(test_df)

In [30]:
print("Accuracy: ", evaluation.accuracy)
print("Precision: ",evaluation.precisionByLabel)
print("Recall: ", evaluation.recallByLabel)

Accuracy:  0.9096118902296976
Precision:  [0.8446654611211574, 0.9321533923303835]
Recall:  [0.8120653685674548, 0.9453249315766024]


### Sbilanciamento delle classi

Se andiamo a guardare la Recall possiamo notare che c'è una notevole differenza fra il valore relativo alle due classi. Questo vuol dire che il modello performa meglio su una classe rispetto a come performa sul'altra; nel caso in questione, su 100 recensioni negative vengono classificate correttamente 80 mentre su 100 positive 94.
In questo caso si parla di **sbilanciamento delle classi**.

Generalmente questo problema è dovuto al fatto che il set di addestramento contiene più esempi appartenenti a una classe rispetto a un'altra.

In [31]:
# Numero esempi di recensioni positive del train set
num_pos_revs = train_df.filter(train_df.label == 1).count()
print(num_pos_revs)

36897


In [32]:
# Numero esempi di recensioni negative del train set
num_neg_revs = train_df.filter(train_df.label == 0).count()
print(num_neg_revs)

13328


Dunque il train set è fortemente sbilanciato a favore degli esempi appartenenti alla classe positiva.

Per rimediare a questo inconveniente la soluzione ideale sarebbe arricchire il dataset con esempi relativi alla classe meno presente; laddove non fosse possibile possiamo **penalizzare la classe verso cui è sbilanciato il dataset assegnandogli un peso**.

Una giusta scelta potrebbe essere quella di scegliere tale peso sulla base del rapporto fra le percentuali di esempi del dataset relativi alle due classi.

In [33]:
pos_weight = float("{:.2f}".format(num_pos_revs/(num_pos_revs + num_neg_revs)))
print(pos_weight)

0.73


In [34]:
neg_weight = 1 - pos_weight
print(neg_weight)

0.27


A questo punto costruisco una nuova colonna nel train set per inserire il peso.

In [35]:
train_df = train_df.withColumn('weight',when(cv_df['stars'] >= 3.5, neg_weight).otherwise(pos_weight))
train_df.show(10)

+--------------------+-----+-----+------+
|            features|stars|label|weight|
+--------------------+-----+-----+------+
|(101222,[0,1,2,3,...|  5.0|    1|  0.27|
|(101222,[0,1,2,3,...|  4.0|    1|  0.27|
|(101222,[0,1,2,3,...|  4.0|    1|  0.27|
|(101222,[0,1,2,3,...|  5.0|    1|  0.27|
|(101222,[0,1,2,3,...|  4.0|    1|  0.27|
|(101222,[0,1,2,3,...|  1.0|    0|  0.73|
|(101222,[0,1,2,3,...|  4.0|    1|  0.27|
|(101222,[0,1,2,3,...|  5.0|    1|  0.27|
|(101222,[0,1,2,3,...|  4.0|    1|  0.27|
|(101222,[0,1,2,3,...|  5.0|    1|  0.27|
+--------------------+-----+-----+------+
only showing top 10 rows



A questo punto rigeneriamo il modello tenendo conto anche della colonna dei pesi tramite il parametro **weightCol**.

In [36]:
lr = LogisticRegression(featuresCol="features", labelCol="label", weightCol = "weight")
model = lr.fit(train_df)

Valutiamo il modello addestrato sul DataFrame di test.

In [37]:
evaluation = model.evaluate(test_df)

In [38]:
print("Accuracy: ", evaluation.accuracy)
print("Precision: ",evaluation.precisionByLabel)
print("Recall: ", evaluation.recallByLabel)

Accuracy:  0.9035549550389042
Precision:  [0.8544474393530997, 0.9192328969205237]
Recall:  [0.771557719054242, 0.9518808478136338]


Andando ad osservare le metriche non possiamo affermare di aver migliorato il modello, nè dal punto di vista dell'accuratezza nè da quello dello sblianciamento delle classi.

Dunque meglio provare con un altro modello (vedi più avanti).

## Creiamo un modello con tutti i dati

Finora abbiamo utilizzato solo l'1% dei dati a nostra disposizione per generare il nostro modello.

Volendo addestrare il modello utilizzando il **dataset per intero** con tutti le sue **6.685.900 recensioni** verrebbero richieste molte risorse di calcolo e, di conseguenza, tempo. Si dovrebbe utilizzare almeno una macchina EC2 di tipo t3a.large (costo ~7 centesimi l'ora) o meglio ancora un cluster con EMR.

## TF-IDF

Proviamo a generare ora un nuovo modello utilizzando, anzichè il Bag of Words, il **TF-IDF** (Term Frequency - Inverse Document Frequency)

In [39]:
reviews_df = yelp_df.select('text','stars')

In [40]:
reviews_df.show(5)

+--------------------+-----+
|                text|stars|
+--------------------+-----+
|As someone who ha...|  2.0|
|I am actually hor...|  1.0|
|I love Deagan's. ...|  5.0|
|Dismal, lukewarm,...|  1.0|
|Oh happy day, fin...|  4.0|
+--------------------+-----+
only showing top 5 rows



Cominciamo creando un modello utilizzando soltanto il 1% del dataset.

In [41]:
subreviews_df = reviews_df.sample(fraction=0.01, seed=0)

### Preprocessing del testo

#### Rimozione della punteggiatura

In [42]:
@udf()
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

subreviews_df = subreviews_df.withColumn('text', remove_punct(col('text')))
subreviews_df.show(10)

+--------------------+-----+
|                text|stars|
+--------------------+-----+
|I have been here ...|  4.0|
|So far one of the...|  5.0|
|Came here for hap...|  1.0|
|Bad 1st experienc...|  1.0|
|So many great thi...|  4.0|
|Great food  Get t...|  4.0|
|Super personable ...|  5.0|
|Clean nice and fr...|  4.0|
|I love their food...|  4.0|
|This hot and read...|  1.0|
+--------------------+-----+
only showing top 10 rows



#### Tokenizzazione

In [43]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(subreviews_df)

In [44]:
words_df.show(5)

+--------------------+-----+--------------------+
|                text|stars|               words|
+--------------------+-----+--------------------+
|I have been here ...|  4.0|[i, have, been, h...|
|So far one of the...|  5.0|[so, far, one, of...|
|Came here for hap...|  1.0|[came, here, for,...|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|
|So many great thi...|  4.0|[so, many, great,...|
+--------------------+-----+--------------------+
only showing top 5 rows



#### Rimozione StopWords

In [45]:
stopwords = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = stopwords.transform(words_df)

In [46]:
words_df.show(5)

+--------------------+-----+--------------------+--------------------+
|                text|stars|               words|            filtered|
+--------------------+-----+--------------------+--------------------+
|I have been here ...|  4.0|[i, have, been, h...|[twice, nice, lai...|
|So far one of the...|  5.0|[so, far, one, of...|[far, one, best, ...|
|Came here for hap...|  1.0|[came, here, for,...|[came, happy, hou...|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|[bad, 1st, experi...|
|So many great thi...|  4.0|[so, many, great,...|[many, great, thi...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



#### TF-IDF

Questa volta, piuttosto che usare un semplice modello Bag of Words per la rappresentazione delle parole, usiamo un modello più sofisticato, il TF-IDF (Term Frequency - Inverse Document Frequency) che assegna un peso maggiore alle parole più rare e penalizza quelle più comuni.
Nello specifico:

* **Term Frequency**:  è il numero di volte in cui un termine appare in un documento (ovvero una riga del dataset)
* **Inverse Document Frequency**: è il numero di documenti (ovvero righe del dataset) che contiene un determinato termine

Anche in questo caso, come succedeva utilzzando le Bags of Words, **Spark** utilizza un approccio che consente di "salvare" spazio ottenendo lo stesso risultato (viene utilizzata una funzione di hash).

In [47]:
from pyspark.ml.feature import HashingTF, IDF

hashing_tf = HashingTF(inputCol='filtered', outputCol='raw_features')
words_df = hashing_tf.transform(words_df)

In [48]:
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(words_df)
words_df = idf_model.transform(words_df)

In [49]:
words_df.show(1,False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [50]:
words_df.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|                text|stars|               words|            filtered|        raw_features|            features|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|I have been here ...|  4.0|[i, have, been, h...|[twice, nice, lai...|(262144,[3856,433...|(262144,[3856,433...|
|So far one of the...|  5.0|[so, far, one, of...|[far, one, best, ...|(262144,[12057,13...|(262144,[12057,13...|
|Came here for hap...|  1.0|[came, here, for,...|[came, happy, hou...|(262144,[1094,343...|(262144,[1094,343...|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|[bad, 1st, experi...|(262144,[25491,32...|(262144,[25491,32...|
|So many great thi...|  4.0|[so, many, great,...|[many, great, thi...|(262144,[329,3837...|(262144,[329,3837...|
+--------------------+-----+--------------------+--------------------+--------------------+-----

A questo punto, come già fatto prima, rimuoviamo le recensioni con 3 stelle e valutiamo positivamente quelle con un numero maggiore di stelle e negativamente tutte le altre.

In [51]:
words_df = words_df.filter(cv_df.stars != 3)
words_df = words_df.withColumn('label',when(cv_df['stars'] >= 3.5, 1).otherwise(0))

In [52]:
words_df.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-----+
|                text|stars|               words|            filtered|        raw_features|            features|label|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-----+
|I have been here ...|  4.0|[i, have, been, h...|[twice, nice, lai...|(262144,[3856,433...|(262144,[3856,433...|    1|
|So far one of the...|  5.0|[so, far, one, of...|[far, one, best, ...|(262144,[12057,13...|(262144,[12057,13...|    1|
|Came here for hap...|  1.0|[came, here, for,...|[came, happy, hou...|(262144,[1094,343...|(262144,[1094,343...|    0|
|Bad 1st experienc...|  1.0|[bad, 1st, experi...|[bad, 1st, experi...|(262144,[25491,32...|(262144,[25491,32...|    0|
|So many great thi...|  4.0|[so, many, great,...|[many, great, thi...|(262144,[329,3837...|(262144,[329,3837...|    1|
+--------------------+-----+--------------------

#### Train set e Test set

Dividiamo il dataframe in set di addestramento e di test, avendo moltissimi esempi possiamo anche ridurre la dimensione del set di test al 10%.

In [53]:
train_df, test_df = words_df.randomSplit([0.9, 0.1])

### Generazione dle modello

Creiamo il modello e addestriamolo.

In [54]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

Valutiamolo sul set di test.

In [55]:
evaluation = model.evaluate(test_df)
print(evaluation.accuracy)
print(evaluation.precisionByLabel)
print(evaluation.recallByLabel)

0.9090035684875103
[0.8404255319148937, 0.9344879518072289]
[0.8266068759342302, 0.9403296078802803]


Il modello sembra essere un po' più performante rispetto al Bag of Words.

## Testo il modello

In [56]:
reviews = [
        ("World's Largest Entertainment McDonald's","Lazy staff who do not want to serve u would rather stand in corners in groups talking stood at counter 10 minutes with all staff refusing eye contact as fear of having to serve u supervisor went over and shouted at staff they all stood there shrugging shoulders not wanting t serve u then when orders were ready staff came with trays dragging feet and rolling eyes then it was cold horrible won’t return !!"),
        ("Disnayland Paris","Went here 2x with my husband and found it more magical the 2nd time. Still the happiest place on earth on my list. It gets better and better."),
        ("58 Tour Eiffel Restaurant","What an experience! what a VIEW!. what a meal!!... Delicious, fine dining. excellent0 excellent service and food. A memory of a lifetime"),
        ("Ristorante Cracco","If you want to start your trip in Milan with good mood, for sure you have to avoid this restaurant - the worst pizza we had and the smallest portion of pasta! And incompatible price for that everything! Even, I am really angry, because this is not my first visit in Italy and not first pizza and I feel myself like ....!!!!"),
        ("Happy Wok","Stay away as far as you can, unless you like goopy tables and mass produced food that appeared to be sitting out for too long. It wasn’t a nice experience and we will not attempt to go back under any circumstance"),
        ("Pepe in grani","45 minutes driving from Naples center. Worth every moment on the way. The best and the most unique pizza I ever tasted. Very nice place, every centimeter was well though and planned before implemented. Nice terrace on top for those like the view. Very welcoming crew, great and fast service. Recommend to order the tasting option for those coming in parties of four. ")
    ]

test_df = spark.createDataFrame(reviews, ["location","text"] )

In [57]:
test_df.show()

+--------------------+--------------------+
|            location|                text|
+--------------------+--------------------+
|World's Largest E...|Lazy staff who do...|
|    Disnayland Paris|Went here 2x with...|
|58 Tour Eiffel Re...|What an experienc...|
|   Ristorante Cracco|If you want to st...|
|           Happy Wok|Stay away as far ...|
|       Pepe in grani|45 minutes drivin...|
+--------------------+--------------------+



In [58]:
test_df = test_df.withColumn("text", remove_punct(col('text')))

In [59]:
test_df.show()

+--------------------+--------------------+
|            location|                text|
+--------------------+--------------------+
|World's Largest E...|Lazy staff who do...|
|    Disnayland Paris|Went here 2x with...|
|58 Tour Eiffel Re...|What an experienc...|
|   Ristorante Cracco|If you want to st...|
|           Happy Wok|Stay away as far ...|
|       Pepe in grani|45 minutes drivin...|
+--------------------+--------------------+



In [60]:
test_df = tokenizer.transform(test_df)
test_df = stopwords.transform(test_df)

In [61]:
test_df.show()

+--------------------+--------------------+--------------------+--------------------+
|            location|                text|               words|            filtered|
+--------------------+--------------------+--------------------+--------------------+
|World's Largest E...|Lazy staff who do...|[lazy, staff, who...|[lazy, staff, wan...|
|    Disnayland Paris|Went here 2x with...|[went, here, 2x, ...|[went, 2x, husban...|
|58 Tour Eiffel Re...|What an experienc...|[what, an, experi...|[experience, view...|
|   Ristorante Cracco|If you want to st...|[if, you, want, t...|[want, start, tri...|
|           Happy Wok|Stay away as far ...|[stay, away, as, ...|[stay, away, far,...|
|       Pepe in grani|45 minutes drivin...|[45, minutes, dri...|[45, minutes, dri...|
+--------------------+--------------------+--------------------+--------------------+



In [62]:
test_df = hashing_tf.transform(test_df)
test_df = idf_model.transform(test_df)

In [63]:
test_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            location|                text|               words|            filtered|        raw_features|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|World's Largest E...|Lazy staff who do...|[lazy, staff, who...|[lazy, staff, wan...|(262144,[3524,844...|(262144,[3524,844...|
|    Disnayland Paris|Went here 2x with...|[went, here, 2x, ...|[went, 2x, husban...|(262144,[14376,31...|(262144,[14376,31...|
|58 Tour Eiffel Re...|What an experienc...|[what, an, experi...|[experience, view...|(262144,[10077,17...|(262144,[10077,17...|
|   Ristorante Cracco|If you want to st...|[if, you, want, t...|[want, start, tri...|(262144,[16004,28...|(262144,[16004,28...|
|           Happy Wok|Stay away as far ...|[stay, away, as, ...|[stay, away, far,...|(262144,[9129,223..

In [64]:
pred_df = model.transform(test_df)

In [65]:
pred_df.select(['location','text','prediction']).show(truncate=False)

+----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|location                                |text                                                                                                                                                                                                                                                                                                                                                                                                                |prediction|
+----------------------------------------+----------------------------------------